In [4]:
import pandas as pd
import numpy as np
#import pcg_skel
import tqdm
#from meshparty import meshwork
from caveclient import CAVEclient
import datetime
import os

client = CAVEclient('minnie65_phase3_v1')

In [5]:
now = client.materialize.get_timestamp()

In [6]:
skeldir = 'skeletons'

In [7]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None

##### EXTENDED FIGURE 5

# Plot for Basket Cells (proximal targeting cells)

In [8]:
# Get synapses formed by the axons of comprehensively extended ET neurons
ET_synapse_table = pd.read_feather("ET_extended_synapse_table_revision.feather")

# Sort postsynaptic basket cells based on the number of synapses that it
# receives from the extended ET neurons
# ----------------------------------------------------------------------

values = []

#root IDs of postynaptic BC neurons
pre_soma_IDs = ET_synapse_table.query('consensus_subclass == "BC"').post_pt_root_id.unique().tolist()


for ii,pre_soma_ID in enumerate(pre_soma_IDs):        
        
   
    stat_values={

                    'ID': pre_soma_ID,
                    'syn_from_ET': len(ET_synapse_table[ (ET_synapse_table['post_pt_root_id'] == pre_soma_ID)]),
             
        
        
    }
    values.append(stat_values)
    
    

synapse_table_BC_values = pd.DataFrame(values)

synapse_table_BC_values_sorted  = synapse_table_BC_values.sort_values(by='syn_from_ET', ascending=False)


In [12]:
# Build the skeletons of the BC neurons

inhibitory_targets_df = synapse_table_BC_values_sorted.head(25) 
print(inhibitory_targets_df)

nrns = {}

for _, row in tqdm.tqdm(inhibitory_targets_df.iterrows()):

    print(row["ID"])
    if os.path.exists(f"{skeldir}/{row['ID']}.h5"):
        nrns[row["ID"]] = meshwork.load_meshwork(f"{skeldir}/{row['ID']}.h5")
   
    else:
    
        nrns[row["ID"]] = pcg_skel.coord_space_meshwork(
            row["ID"],
            client=client,
            root_point= ET_synapse_table[ET_synapse_table['post_pt_root_id'] == row["ID"]].post_soma_pt.iloc[0],#row["pt_position"], post_soma_pt
            root_point_resolution=[4, 4, 40],
            collapse_soma=True,
            synapses="all",
            synapse_table=client.info.get_datastack_info().get("synapse_table"),
            timestamp = now,
        )

        nrns[row["ID"]].save_meshwork(f"{skeldir}/{row['ID']}.h5")        
        
# Get the axons
for rid, nrn in nrns.items():
    is_axon = meshwork.algorithms.split_axon_by_annotation(
        nrn,
        'pre_syn',
        'post_syn',
        return_quality=False
    )
    nrn.anno.add_annotations('is_axon', is_axon, mask=True)

                     ID  syn_from_ET
65   864691137198630977           35
6    864691135066377796           33
83   864691136092188340           32
171  864691135697251482           27
104  864691135885194352           22
188  864691136021592568           21
39   864691135276293349           21
91   864691135640274491           19
220  864691136024968505           19
41   864691136175460102           19
55   864691135772785915           18
105  864691136109199800           18
29   864691135594101291           18
191  864691135993185985           17
225  864691135865971164           16
217  864691136005978826           16
102  864691135773256443           15
1    864691135386608213           15
119  864691135463586973           14
110  864691135463999294           14
263  864691136966961614           13
101  864691136176529158           12
11   864691136723943421           12
213  864691135360007512           12
185  864691135947023969           12


0it [00:00, ?it/s]

864691137198630977


NameError: name 'pcg_skel' is not defined

In [13]:
#Filter for presynaptic outputs on target cell axons and concatenate into one dataframe, adding synapse distance:

pre_dfs = []
for rid in inhibitory_targets_df["ID"]:
    syn_filt = nrns[rid].anno.pre_syn.filter_query(
            nrns[rid].anno.is_axon.mesh_mask
    )
    df = syn_filt.df
    df['dist_to_root'] = nrns[rid].distance_to_root(syn_filt.mesh_index)
    df['distance_rank'] = df['dist_to_root'].rank()
    df.attrs = {}
    pre_dfs.append(df)

inhibitory_targets_pre_df= pd.concat(pre_dfs, ignore_index=True)
inhibitory_targets_pre_df['pre_pt_root_id'] = client.chunkedgraph.get_roots(inhibitory_targets_pre_df['pre_pt_supervoxel_id'], timestamp=now)
inhibitory_targets_pre_df['post_pt_root_id'] = client.chunkedgraph.get_roots(inhibitory_targets_pre_df['post_pt_supervoxel_id'], timestamp=now).astype('int')


KeyError: 864691137198630977

In [ ]:
# Get single soma root ids and add cell types

soma_df = client.materialize.query_table(
    "nucleus_neuron_svm", filter_equal_dict={"cell_type": "neuron"}
)

soma_df = soma_df.drop_duplicates(subset="pt_root_id", keep='first')

mtypes_model_df = client.materialize.query_table(
    "aibs_metamodel_mtypes_v661_v2",
).drop_duplicates('pt_root_id', keep=False)

# Enrich soma_df with all this info
soma_df = (
    soma_df.merge(        
        mtypes_model_df[["pt_root_id", "cell_type"]].rename(
            columns={"cell_type": "model_cell_type"}
        ),
        on="pt_root_id",
        how="left",
    )
)


#Add subclass labels to soma_df
def standardize_subclass(row):
        
        
    if row['model_cell_type'] == 'L3b':
          return 'L23-P'

    if row['model_cell_type'] == 'MC':
          return 'DTC'
    
    if row['model_cell_type'] == 'BC':
          return 'PTC'
    
    if row['model_cell_type'] == 'BPC':
          return 'ITC'
    
    if row['model_cell_type'] == 'NGC':
          return 'STC'
            
    if row['model_cell_type'] == '23P':
          return 'L23-P'
            
    if row['model_cell_type'] == 'L2b':
          return 'L23-P'
          
    if row['model_cell_type'] == 'L6tall-c':
          return 'L6-P'
          
    if row['model_cell_type'] == 'L6short-b':
          return 'L6-P'
          
    if row['model_cell_type'] == 'L4b':
          return 'L4-P'
          
    if row['model_cell_type'] == 'L2a':
          return 'L23-P'
   
    if row['model_cell_type'] == 'L3a':
          return 'L23-P'
          
    if row['model_cell_type'] == 'L6tall-b':
          return 'L6-P'
          
    if row['model_cell_type'] == 'L2c':
          return 'L23-P'
          
    if row['model_cell_type'] == 'L4a':
          return 'L4-P'

    if row['model_cell_type'] == '4P':
          return 'L4-P'
        
    if row['model_cell_type'] == 'L6wm':
          return 'L6-P'
        
    if row['model_cell_type'] == 'L6tall-a':
          return 'L6-P'
 
    if row['model_cell_type'] == 'L6short-a':
          return 'L6-P'
          
    if row['model_cell_type'] == 'L5a':
          return 'L5-IT'
        
    if row['model_cell_type'] == 'L5b':
          return 'L5-IT'
        
    if row['model_cell_type'] == 'L5ET':
          return 'L5-ET'

    if row['model_cell_type'] == 'L5NP':
          return 'L5-NP'
  
    if row['model_cell_type'] == 'L4c':
          return 'L4-P'
        
    if row['model_cell_type'] == 'none':
          return None
               
    if pd.isna(row['model_cell_type']) == True:
          return None
        
    else:
          return row['model_cell_type'] 

soma_df['model_subclass'] = soma_df.apply(standardize_subclass, axis=1)


#Merge all this info from cell types into the synapse dataframe, as well as add area locations.

#merge presynaptic nucleous ID
synapse_table = inhibitory_targets_pre_df.merge(
    soma_df[
        ["id", "pt_root_id", "pt_position", "model_subclass"]
    ].rename(columns={"pt_position": "post_soma_pt"}).rename(columns={"id": "post_nucleus_id"}),
    left_on="post_pt_root_id",
    right_on="pt_root_id",
    how="left",
).drop(columns="pt_root_id")

synapse_table = synapse_table.rename(columns={"id": "synapse_id"})

In [ ]:
values = []

#root IDs of presynaptic PT neurons
pre_soma_IDs = inhibitory_targets_df.ID.tolist()


for ii,pre_soma_ID in enumerate(pre_soma_IDs):        
        
    #print(pre_soma_ID)
    stat_values={

                    'ID': pre_soma_ID,
                            
                    #SYNAPSES
        
                    
                    'all_total_syn#': len(synapse_table[ (synapse_table['pre_pt_root_id'] == pre_soma_ID)]),
        
                    'all_total_syn#_with_prediction': len(synapse_table[synapse_table['pre_pt_root_id'] == pre_soma_ID]
                                                          ['model_subclass'].dropna()), 
                    
                    'all_total_L5_targets_syn#': len(synapse_table[ ((synapse_table['model_subclass'] == 'L5-ET') |
                                                    (synapse_table['model_subclass'] == 'L5-IT') |
                                                    (synapse_table['model_subclass'] == 'L5-NP')) &
                                                    (synapse_table['pre_pt_root_id'] == pre_soma_ID)]),
        
                     
                    'L5-ET_syn#': len(synapse_table[ (synapse_table['model_subclass'] == 'L5-ET') &
                                                    (synapse_table['pre_pt_root_id'] == pre_soma_ID)]),
        
                    
                    'fraction_syn_L5-ET': len(synapse_table[ (synapse_table['model_subclass'] == 'L5-ET') &
                                                    (synapse_table['pre_pt_root_id'] == pre_soma_ID)]) /
                                            len(synapse_table[synapse_table['pre_pt_root_id'] == pre_soma_ID]
                                                          ['model_subclass'].dropna()),
        
                    'fraction_syn_L5-ET_from_L5': len(synapse_table[ (synapse_table['model_subclass'] == 'L5-ET') &
                                                    (synapse_table['pre_pt_root_id'] == pre_soma_ID)]) /
                                            len(synapse_table[ ((synapse_table['model_subclass'] == 'L5-ET') |
                                                    (synapse_table['model_subclass'] == 'L5-IT') |
                                                    (synapse_table['model_subclass'] == 'L5-NP')) &
                                                    (synapse_table['pre_pt_root_id'] == pre_soma_ID)]),
               
                    'fraction_syn_L5-ET_from_Exc': len(synapse_table[ (synapse_table['model_subclass'] == 'L5-ET') &
                                                    (synapse_table['pre_pt_root_id'] == pre_soma_ID)]) /
                                            len(synapse_table[ ((synapse_table['model_subclass'] == 'L5-ET') |
                                                    (synapse_table['model_subclass'] == 'L5-IT') |
                                                    (synapse_table['model_subclass'] == 'L5-NP')|
                                                    (synapse_table['model_subclass'] == 'L23-P')|
                                                    (synapse_table['model_subclass'] == 'L4-P')|
                                                    (synapse_table['model_subclass'] == 'L6-P'))&
                                                    (synapse_table['pre_pt_root_id'] == pre_soma_ID)]),
        
                   

        
        
    }
    values.append(stat_values)
    
    

synapse_area_table_values = pd.DataFrame(values)

synapse_area_table_values

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

f, ax = plt.subplots(figsize=(8,4))

ax = sns.swarmplot(data=synapse_area_table_values, x="fraction_syn_L5-ET_from_Exc", size=10)

ax.set(xlim=(0, 1))
ax.set_xticks([0,0.25,0.5,0.75,1])

plt.savefig('Extended_5.eps')